In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter
import json
import numpy as np

In [24]:
with open("data/output/사업계획서자손.json", "r", encoding="UTF-8") as fp:
    text= fp.read()
    
json_data = json.loads(text)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
docs = splitter.create_documents(texts=[json_data], ensure_ascii=False)
#docs = splitter.split_json(json_data=json_data)

In [13]:
docs[40:50]

[Document(page_content='{"2023": {"자본": {"자본총계": {"Value": {"Samsung": 363677865, "Hyundai": 101809440, "Kepco": 37264750, "Sum": 502752055, "Avg": 167584018.33333334}}}}}'),
 Document(page_content='{"2023": {"자본": {"지배기업 소유주지분": {"Value": {"Samsung": 353233775, "Hyundai": "-", "Kepco": "-", "Sum": 353233775, "Avg": 353233775.0}}}}}'),
 Document(page_content='{"2023": {"자본": {"지배기업 소유주지분": {"기타자본항목": {"Value": {"Samsung": 1280130, "Hyundai": "-", "Kepco": "-", "Sum": 1280130, "Avg": 1280130.0}}}}}}'),
 Document(page_content='{"2023": {"자본": {"지배기업 소유주지분": {"이익잉여금": {"Value": {"Samsung": 346652238, "Hyundai": "-", "Kepco": "-", "Sum": 346652238, "Avg": 346652238.0}}}}}}'),
 Document(page_content='{"2023": {"자본": {"지배기업 소유주지분": {"자본금": {"Value": {"Samsung": 897514, "Hyundai": "-", "Kepco": "-", "Sum": 897514, "Avg": 897514.0}}}}}}'),
 Document(page_content='{"2023": {"자본": {"지배기업 소유주지분": {"자본금": {"보통주자본금": {"Value": {"Samsung": 778047, "Hyundai": "-", "Kepco": "-", "Sum": 778047, "Avg": 

In [16]:
type(docs[0].page_content)

str

In [18]:
doc_texts = []
for idx in range(len(docs)):
    doc_texts.append(docs[idx].page_content)

In [19]:
doc_texts[0]

'{"2023": {"부채": {"Value": {"Samsung": "-", "Hyundai": "-", "Kepco": "-", "Sum": "-", "Avg": "-"}, "부채총계": {"Value": {"Samsung": 92228115, "Hyundai": 180653915, "Kepco": 202450215, "Sum": 475332245, "Avg": 158444081.66666666}}}}}'

In [20]:
from langchain_openai import OpenAIEmbeddings
text_embedding_model = OpenAIEmbeddings()

embedded_texts = text_embedding_model.embed_documents(doc_texts)

In [25]:
embedded_query = text_embedding_model.embed_query("기타자본항목")
embedded_query_np = np.array(embedded_query)
embedded_query_reshaped = embedded_query_np.reshape(1, -1)

In [28]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_texts(
    doc_texts,
    text_embedding_model,
    collection_name = "saup",
    persist_directory = "data/output/chromadb",
    collection_metadata = {"hnsw:space":"cosine"}       # 이거 뭘까 공부할 필요가 있음
)

In [37]:
query = "영업이익"
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs={"k":5, "fetch_k":50}
)
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content)

{"2023": {"영업이익": {"Value": {"Samsung": 6566976, "Hyundai": 15126901, "Sum": 21693877, "Avg": 10846938.5}}}}
{"2021": {"주당이익": {"기본주당이익": {"중단영업 보통주기본주당이익(손실)": {"Value": {"Samsung": "-", "Hyundai": 0.000458, "Sum": 0.000458, "Avg": 0.000458}}}}}}
{"2022": {"자본": {"지배기업의 소유주에게 귀속되는 자본": {"이익잉여금(결손금)": {"미처분이익잉여금(미처리결손금)": {"Value": {"Samsung": "-", "Hyundai": "-", "Kepco": -7956579, "Sum": -7956579, "Avg": -7956579.0}}}}}}}
{"2023": {"관계기업,공동기업,종속기업 관련이익(손실)": {"관계기업 및 공동기업 투자지분 평가이익": {"Value": {"Samsung": "-", "Hyundai": "-", "Kepco": 804141, "Sum": 804141, "Avg": 804141.0}}}}}
{"2022": {"영업이익(손실)": {"Value": {"Samsung": "-", "Hyundai": "-", "Kepco": -32655153, "Sum": -32655153, "Avg": -32655153.0}}}}


### 일단 잘 안나온다... 구조를 바꿀 필요가 있어 보인다.
### 제일 먼저 드는 생각은 최하위 이름을 앞으로 내밀고 상위계층의 이름을 원소로 만들어 리스트 형식으로 나열하는건 어떨까